# GCC's ssc-cdi : Tomography Interface for Cateretê, v0.2.1
 
- #### Run first cell to connect to server and install requirements. Run second cell to deploy interface.
- #### To visualize outputs in real time, open the log console at: View > Show Log Console
- #### Questions? E-mail gcc@lnls.br
- #### Installation and User guide [available here](https://10.10.10.141/ssc/wiki/). 

In [1]:
%matplotlib widget
import sscInstall
server = sscInstall.connect_server()
# sscInstall.install_jupyter(server,beamline='caterete',force_install=False,vCdi='0.1.13',vPtycho='1.0.2',vPimega='0.0.6',vRaft='1.0.3',vRadon='1.0.1',vResolution='1.2.3')
from sscCdi import jupyter, tomo_jupyter

Username: yuri.tonin


Password:


 ············


/home/ABTLUS/yuri.tonin/.local/lib/python3.6/site-packages/sscCdi-0.1.14-py3.6-linux-ppc64le.egg/sscCdi
to be done!
/home/ABTLUS/yuri.tonin/.local/lib/python3.6/site-packages/sscRaft/lib/libraft.cpython-36m-powerpc64le-linux-gnu.so


In [2]:
box,tabs, global_dict = tomo_jupyter.deploy_tabs(server)
display(box)
tabs

In [7]:
global_dict

{'jupyter_folder': '/ibira/lnls/beamlines/caterete/apps/jupyter/',
 'ibira_data_path': '/ibira/lnls/beamlines/caterete/proposals/20210127/data/ptycho3d/',
 'folders_list': ['svwt_candb_mem2_01', 'svwt_candb_mem2_02'],
 'sinogram_path': '/ibira/lnls/labs/tepui/proposals/20210062/yuri/yuri_ssc-cdi/00000000/proc/recons/svwt_candb_mem2_01/svwt_candb_mem2_01_object.npy',
 'processing_steps': {'Sort': False,
  'Crop': False,
  'Unwrap': False,
  'ConvexHull': False,
  'Wiggle': False,
  'Tomo': False,
  'Equalize Frames': False,
  'Equalize Recon': False},
 'contrast_type': 'Phase',
 'top_crop': 0,
 'bottom_crop': 1,
 'left_crop': 0,
 'right_crop': 1,
 'bad_frames_before_unwrap': [],
 'unwrap_iterations': 0,
 'unwrap_non_negativity': False,
 'unwrap_gradient_removal': False,
 'equalize_invert': False,
 'equalize_gradient': 0,
 'equalize_outliers': 0,
 'equalize_global_offset': False,
 'equalize_local_offset': [0, None, 0, None],
 'bad_frames_before_cHull': [],
 'chull_invert': False,
 'chull

In [3]:
import numpy as np
import matplotlib.pyplot as plt

def pad_sinogram_frames(padding,path):
    sinogram = np.load(path)
    print("Old shape: ",sinogram.shape)
    sinogram = np.pad(sinogram,((0,0),(padding,padding),(padding,padding)))
    print("New shape: ",sinogram.shape)
    # np.save(path,sinogram)
    return sinogram

path = "/ibira/lnls/beamlines/caterete/proposals/20210127/proc/recons/svwt_candb_mem2_01/Phase_equalized_sinogram.npy"
data = pad_sinogram_frames(100,path)

def gaus2d(x=0, y=0, mx=0, my=0, sx=1, sy=1):
    return 1. / (2. * np.pi * sx * sy) * np.exp(-((x - mx)**2. / (2. * sx**2.) + (y - my)**2. / (2. * sy**2.)))

# path = "/ibira/lnls/beamlines/caterete/proposals/20210127/proc/recons/svwt_candb_mem2_01/Phase_equalized_sinogram.npy"
# data = np.load(path)

sigma = 1.5

x = np.linspace(-5, 5,data.shape[2])
y = np.linspace(-5, 5,data.shape[1])
x, y = np.meshgrid(x, y) # get 2D variables instead of 1D
border_attenuation_matrix = gaus2d(x, y,sx=sigma,sy=sigma)
for i in range(data.shape[0]):
    if i%50==0: print(i,data.shape[0])
    data[i] = border_attenuation_matrix*data[i]

# path = "/ibira/lnls/beamlines/caterete/proposals/20210127/proc/recons/svwt_candb_mem2_01/Phase_equalized_sinogram_filtered.npy"
np.save(path,data)
print("Done")

Old shape:  (400, 662, 1054)
New shape:  (400, 862, 1254)
0 400
50 400
100 400
150 400
200 400
250 400
300 400
350 400
Done


In [6]:
path = "/ibira/lnls/beamlines/caterete/proposals/20210127/proc/recons/svwt_candb_mem2_01/Phase_svwt_candb_mem2_01_reconstruction3D_EEM.npy"
path = "/ibira/lnls/beamlines/caterete/proposals/20210127/proc/recons/svwt_candb_mem2_01/Phase_wiggle_sinogram.npy"

data = np.load(path)

data.shape

(813, 862, 1254)

In [3]:
import numpy as np
path = "/ibira/lnls/beamlines/caterete/proposals/20210127/proc/recons/svwt_candb_mem2_01/Phase_equalized_sinogram.npy"
data = np.load(path)[0:50]

In [17]:
import numpy
def get_extrapolated_sinogram(sino, pad, L, R):
    if pad==0:
        return sino
    else:
        nrays = sino.shape[0]
        nangles = sino.shape[1]
        n = nrays + 2*pad
        s = numpy.vstack((numpy.zeros([pad,nangles]),sino,numpy.zeros([pad,nangles])) )
        t = numpy.linspace(-1,1,n)
        left = (1.0*(( t < t[pad] ) )) 
        right =  1.0 * (t > t[nrays+pad-1])
        L_ = 1
        R_ = 1
        p = numpy.array( [ s[:,j] + L_*s[pad,j]*numpy.exp(L*(t-t[pad]))*left + R_*s[nrays+pad-1,j]*right*numpy.exp(-R*(t-t[nrays+pad-1])) for j in range(nangles)] ).T
        return p
    
size_rays = data.shape[1]*data.shape[2]
n_angles = data.shape[0]

print(data.shape[1],data.shape[2])

sino = np.zeros((size_rays,n_angles))
    
for i in range(data.shape[0]):
    if i+1%10==0: print(i)
    sino[:,i] = data[i,:,:].flatten()

L    = 10
R    = 10
pad  = 150
sino_ext = get_extrapolated_sinogram( sino, pad, L, R)

print(sino.shape,sino_ext.shape,sino.shape[0]-sino_ext.shape[0])

print(sino_ext.shape[0]/(data.shape[1]-pad))

new_shape = (int(sino_ext.shape[0]/(data.shape[1]-pad)),int(sino_ext.shape[0]/(data.shape[2]-pad)),sino_ext.shape[1])
print("New shape", new_shape)
sino_reshaped = np.reshape(sino_ext, new_shape)

print(sino_reshaped.shape)

# import matplotlib.pyplot as plt
# fig, ax = plt.subplots(1,2)
# ax1,ax2 = ax.ravel()
# ax1.imshow(sino)
# ax2.imshow(sino_ext)

862 1254
(1080948, 50) (1081248, 50) -300
1518.6067415730338
New shape (1518, 979, 50)


ValueError: cannot reshape array of size 54062400 into shape (1518,979,50)

In [3]:
import numpy as np
data = np.ones((15,100,100))

path = "/ibira/lnls/beamlines/caterete/proposals/20210127/proc/recons/svwt_candb_mem2_01/Phase_svwt_candb_mem2_01_reconstruction3D_EEM_original.npy"
# path = "/ibira/lnls/beamlines/caterete/proposals/20210127/proc/recons/svwt_candb_mem2_01/Phase_svwt_candb_mem2_01_reconstruction3D_EEM.npy"

data = np.load(path)

data = data[::4,::4,::4]
# data = data[:,::2,::2]
    
path = "/ibira/lnls/beamlines/caterete/proposals/20210127/proc/recons/svwt_candb_mem2_01/Phase_svwt_candb_mem2_01_reconstruction3D_EEM.npy"

np.save(path,data)